# National Water Model Reanalysis v2.1 Assessment
* The National Water Model Reanalysis v2.1 is available as a Zarr dataset on AWS Public Dataset on us-east-1. 
* For speed we use a derivative rechunked product created by James McCreight where each chunk is a full time series at a feature_id
* Extract data at benchmark locations for assessment compared to observed streamflow. 
* Average hourly NWM data to daily
* interpolate obs and data onto time base
* use xskillscore to compare

In [ ]:
import pandas as pd
import xarray as xr
import fsspec
import hvplot.xarray
import cf_xarray
import cf_xarray.units  # must be imported before pint_xarray
import pint_xarray
from pint_xarray import unit_registry as ureg
import xskillscore as xs
from pathlib import Path

#### Open CSV file with obs info from Sydney Foks

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
url = 's3://esip-qhub-public/usgs/hytest/streamflow_benchmark_sites_v09.csv'

In [ ]:
fs.size(url)

In [ ]:
df = pd.read_csv(fs.open(url), dtype={'site_no':str, 'huc_cd':str, 'reachcode':str, 'comid':str})

In [ ]:
df.info()

In [ ]:
df_test = df.head()

In [ ]:
df_test

#### pick some sites

In [ ]:
site_ids = list(df_test['site_no'].values)
site_ids

#### pick a time period

In [ ]:
start = '1995-01-01'
stop = '1995-12-19'

In [ ]:
#National Water Model
url = 's3://noaa-nwm-retrospective-2-1-zarr-pds'

In [ ]:
flist = fs.ls(url)
flist

In [ ]:
%%time
ds = xr.open_dataset(fs.get_mapper(flist[0]), engine='zarr', backend_kwargs={'consolidated':True}, chunks={})

In [ ]:
ds

In [ ]:
ds.streamflow

#### Use a rechunked timeseries datasets to speed things up
We could extract time series using the above zarr dataset, but it would be a bit slow, as many chunks would need to be read for each time series extraction.   Instead, we use another dataset created by James McCreight at NCAR, which rechunked just the streamflow and velocity data at the 5000 gauge locations and rechunked them so that each time series is a single chunk

In [ ]:
fs2 = fsspec.filesystem('s3', requester_pays=True, profile='esip-qhub')

In [ ]:
fs2.ls('s3://esip-qhub/ncar/nwm')

In [ ]:
url = 's3://esip-qhub/ncar/nwm/chanobs.zarr'

In [ ]:
ds_chanobs = xr.open_dataset(fs2.get_mapper(url), engine='zarr', backend_kwargs={'consolidated':False}, chunks={})

In [ ]:
ds_chanobs.nbytes/1e9

In [ ]:
ds_chanobs

In [ ]:
ds_chanobs.streamflow

In [ ]:
def mod_station(ds, site_id):
    ds_sta = ds.where(ds.gage_id == f'{site_id.rjust(15, " ")}'.encode(), drop=True)
    return ds_sta

In [ ]:
%%time 
mod = []
for site_id in site_ids:
    mod.append(mod_station(ds_chanobs, site_id))

In [ ]:
ds_mod = xr.concat(mod, dim='feature_id')

In [ ]:
ds_mod = ds_mod.rename({'streamflow':'discharge', 'feature_id':'station_id'})

#### Extract obs data using hyriver

In [ ]:
from pygeohydro import NWIS

In [ ]:
nwis = NWIS()

In [ ]:
start = ds_mod.time[0].values
stop = ds_mod.time[-1].values

In [ ]:
dates = (start, stop)
dates

In [ ]:
# if changing start,stop time or number of stations, remove the NWIS netcdf file
# fs_local = fsspec.filesystem('file')
#fs_local.rm('nwis.nc')

In [ ]:
%%time
ncfile = 'nwis.nc'

nc_file = Path(ncfile)

if nc_file.is_file():
    ds_obs = xr.open_dataset(ncfile)
else:
    ds_obs = nwis.get_streamflow(site_ids, dates, to_xarray=True)
    ds_obs.to_netcdf(ncfile) 

#### loop over stations

In [ ]:
start = '1995-01-01'
stop = '1995-05-01'

In [ ]:
skill_time_range = pd.date_range('1995-01-01','1995-05-01',freq='1D')

average hourly model output to daily before interpolation:

In [ ]:
%%time
ds_mod_daily = (ds_mod['discharge'].
    sel(time=slice(start,stop)).
    resample(time='1D').mean().
    interp(time=skill_time_range))

Don't need to average daily obs to daily before interpolation:

In [ ]:
ds_obs_daily = ds_obs['discharge'].interp(time=skill_time_range)

#### comparison plot at a specific site_id

In [ ]:
ds_mod_daily.isel(station_id=0)

In [ ]:
def comp_plot(site_id):
    obs = ds_obs_daily.sel(station_id=f'USGS-{site_id}')
    mod = mod_station(ds_mod_daily, site_id)
    plt = obs.hvplot(x='time',label='observed') * mod.hvplot(x='time', label='data')
    display(plt.opts(show_grid=True))

In [ ]:
comp_plot(site_ids[0])

#### compute the pearson correlation using xskillscore

Xskillscore has a large number of [deterministic](https://xskillscore.readthedocs.io/en/stable/quick-start.html#Deterministic-Metrics) and [probabalistic](https://xskillscore.readthedocs.io/en/stable/quick-start.html#Probabilistic-Metrics) metrics

In [ ]:
r = xs.pearson_r(ds_obs_daily,ds_mod_daily, dim='time')

In [ ]:
r